In [0]:
!pip install ufal.udpipe

from ufal.udpipe import Model, Pipeline
from nltk import DependencyGraph
import json
import os
from pymystem3 import Mystem
mystem = Mystem() 

UDPIPE_MODEL_FN = "model_ru.udpipe"
!wget -O {UDPIPE_MODEL_FN} https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe?raw=true

model = Model.load(UDPIPE_MODEL_FN)

     |████████████████████████████████| 307kB 4.8MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp36-cp36m-linux_x86_64.whl size=5625898 sha256=815666f894ef35189edebd5ef5cc5fc3919ce93c7ec96e736b2bd7a1b0a0c32c
  Stored in directory: /root/.cache/pip/wheels/0c/9d/db/6d3404c33da5b7adb6c6972853efb6a27649d3ba15f7e9bebb
Successfully built ufal.udpipe


Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


--2020-01-18 10:09:44--  https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe?raw=true
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/jwijffels/udpipe.models.ud.2.0/raw/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe [following]
--2020-01-18 10:09:44--  https://github.com/jwijffels/udpipe.models.ud.2.0/raw/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jwijffels/udpipe.models.ud.2.0/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe [following]
--2020-01-18 10:09:44--  https://raw.githubusercontent.com/jwijffels/udpipe.models.ud.2.0/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpip

In [0]:
from google.colab import files
files.upload()

Saving anthems.json to anthems.json


{'anthems.json': b'{\r\n    "0": {\r\n        "name": "\xd0\x90\xd0\xb1\xd1\x85\xd0\xb0\xd0\xb7\xd0\xb8\xd1\x8f",\r\n        "link": "http://discoveric.ru/anthem/abhaziya",\r\n        "text": "\xd0\x92\xd0\xbf\xd0\xb5\xd1\x80\xd0\xb5\xd0\xb4, \xd1\x81\xd1\x8b\xd0\xbd\xd1\x8b \xd0\x90\xd0\xb1\xd1\x85\xd0\xb0\xd0\xb7\xd0\xb8\xd0\xb8!!!\\n\xd0\xa1\xd0\xb2\xd1\x8f\xd1\x82\xd0\xb0\xd1\x8f \xd0\xba\xd1\x80\xd0\xbe\xd0\xb2\xd1\x8c, \xd0\xbf\xd1\x80\xd0\xbe\xd0\xbb\xd0\xb8\xd1\x82\xd0\xb0\xd1\x8f \xd0\xb7\xd0\xb0 \xd0\xa0\xd0\xbe\xd0\xb4\xd0\xb8\xd0\xbd\xd1\x83,\\n\xd0\x9d\xd0\xb0\xd1\x88\xd0\xb8 \xd0\xbf\xd1\x80\xd0\xb5\xd0\xb4\xd0\xba\xd0\xb8 \xd0\xbf\xd1\x80\xd0\xbe\xd0\xbb\xd0\xb8\xd0\xb2\xd0\xb0\xd0\xbb\xd0\xb8 \xd0\xb7\xd0\xb0\\n\xd0\x9d\xd0\xb0\xd1\x88\xd0\xb8\xd1\x85 \xd1\x81\xd1\x8b\xd0\xbd\xd0\xbe\xd0\xb2\xd0\xb5\xd0\xb9!\\n\xd0\x91\xd0\xb5\xd0\xb7 \xd1\x81\xd0\xbe\xd0\xbb\xd0\xbd\xd1\x86\xd0\xb0 \xd0\xbd\xd0\xb5\xd0\xb1\xd0\xbe \xd0\xbd\xd0\xb5\xd1\x82,\\n\xd0\xa1\xd0\xb5\xd1\x80\xd

In [0]:
with open('anthems.json', 'r', encoding='utf-8') as f:
    anthems = json.loads(f.read(), object_hook=lambda d: {int(k) if k.lstrip('-').isdigit() else k: v for k, v in d.items()})

In [0]:
def parse(text):

  pipeline = Pipeline(model, 'generic_tokenizer', '','','') #функции нужно 5 аргументов,но нам важны только 2

  ud_res = []
  parsed_text = pipeline.process(text) # функция process сделает синтаксический анализ, сохраняем еще раз

  return parsed_text

def conllu_to_list(parser_result): 
    sents = []
    for sent in parser_result.split('\n\n')[:-1]:
        sent = '\n'.join([line for line in sent.split('\n') if not line.startswith('#')])
        sent = sent.replace('\troot\t', '\tROOT\t')
        sents.append(sent)
    return sents

def get_ud_graphs(sentences):
  ud_graphs = []
  for sentence in sentences:
    ud_graph = DependencyGraph(tree_str=sentence)
    ud_graphs.append(ud_graph)
  return ud_graphs

In [0]:
for anthem in anthems.values():
  text = anthem['text']
  anthem['ud_graphs'] = get_ud_graphs(conllu_to_list(parse(text))) # sentences = graphs
  sentences = []
  for ud_graph in anthem['ud_graphs']: # for sentence (= set of tokens = graph = dict) in sentences
    tokens = []
    n_tokens = len(ud_graph.nodes) # how many tokens in sentence = keys (0, 1, 2...) in dict
    for i in range(0, n_tokens): # go through every token = dict
      d = dict(ud_graph.nodes[i]) # all tokens in sentence as dict
      deps = dict(d['deps']) # convert 'deps' into dict
      d['deps'] = deps
      tokens.append(d)
    sentences.append(tokens)
  anthem['ud_graphs'] = sentences

In [0]:
anthems[79]

{'area': 300000,
 'continent': 'AS',
 'former_colony': 1,
 'link': 'http://discoveric.ru/anthem/filippiny',
 'name': 'Филиппины',
 'name_en': 'Philippines',
 'population': 100981437,
 'religion': {'christianity': 91.3, 'islam': 5.5, 'religion': 3.2},
 'secular': 1,
 'state_characteristics': ['unitary state', 'presidential system', 'republic'],
 'text': 'Любимая земля,\nЖемчужина Востока,\nОгонь сердца,\nБьющийся в каждой груди.\n\nИзбранная земля\nТы - колыбель мужества,\nЗахватчикам\nНе сдающаяся никогда.\n\nВ морях и в горах,\nВ воздухе и в голубом небе\nВеликолепие в стихах\nИ песнях о желанной свободе.\n\nВ сиянье твоего флага\nГорит победа,\nЕго звезды и солнце\nДа не померкнут никогда\n\nЗемля солнца, славы и нашей любви,\nЖизнь - это рай в твоих объятьях.\nНа нашу радость существуют угнетатели:\nМы можем умереть за тебя.',
 'text_lemmatized': 'любимый земля,\nжемчужина восток,\nогонь сердце,\nбиться в каждый грудь.\n\nизбранный земля\nты - колыбель мужество,\nзахватчик\nне сдава

In [0]:
with open('anthems.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(anthems, ensure_ascii=False, indent=4)) 

In [0]:
files.download('anthems.json')